In [1]:
# import libraries
import os, sys
import sys
import random
import math
import numpy as np
import scipy.misc
import coco
import mrcnn.model as modellib
from mrcnn import utils, visualize
import matplotlib
import matplotlib.pyplot as plt

ModuleNotFoundError: No module named 'mrcnn'

In [3]:
import tensorflow as tf
print(tf.__version__)

2.5.0


In [3]:
import os
import sys
import coco
from mrcnn import utils 
from mrcnn import model as modellib
from mrcnn.config import Config

# We use a K80 GPU with 24GB memory, which can fit 3 images.
# batch_size = 3

ROOT_DIR = os.path.abspath("../")
print(ROOT_DIR)

C:\Users\liewz\Mask_RCNN


In [4]:
MODEL_DIR = os.path.join(ROOT_DIR, "logs")
VIDEO_DIR = os.path.join(ROOT_DIR, "Image_download_v2_post1-10000")
VIDEO_SAVE_DIR = os.path.join(VIDEO_DIR, "save")
COCO_MODEL_PATH = os.path.join(ROOT_DIR, "mask_rcnn_coco.h5")
if not os.path.exists(COCO_MODEL_PATH):
    utils.download_trained_weights(COCO_MODEL_PATH)

class CocoConfig(Config):
    """Configuration for training on MS COCO.
    Derives from the base Config class and overrides values specific
    to the COCO dataset.
    """
    # Give the configuration a recognizable name
    NAME = "coco"

    # We use a GPU with 12GB memory, which can fit two images.
    # Adjust down if you use a smaller GPU.
    IMAGES_PER_GPU = 1

    # Uncomment to train on 8 GPUs (default is 1)
    # GPU_COUNT = 8

    # Number of classes (including background)
    NUM_CLASSES = 1 + 80  # COCO has 80 classes

class InferenceConfig(CocoConfig):
#class InferenceConfig(Config):
    # Set batch size to 1 since we'll be running inference on
    # one image at a time. Batch size = GPU_COUNT * IMAGES_PER_GPU
    GPU_COUNT = 1
    IMAGES_PER_GPU = 1

config = InferenceConfig()
config.display()

model = modellib.MaskRCNN(
    mode="inference", model_dir=MODEL_DIR, config=config
)
model.load_weights(COCO_MODEL_PATH, by_name=True)
class_names = [
    'BG', 'person', 'bicycle', 'car', 'motorcycle', 'airplane',
    'bus', 'train', 'truck', 'boat', 'traffic light',
    'fire hydrant', 'stop sign', 'parking meter', 'bench', 'bird',
    'cat', 'dog', 'horse', 'sheep', 'cow', 'elephant', 'bear',
    'zebra', 'giraffe', 'backpack', 'umbrella', 'handbag', 'tie',
    'suitcase', 'frisbee', 'skis', 'snowboard', 'sports ball',
    'kite', 'baseball bat', 'baseball glove', 'skateboard',
    'surfboard', 'tennis racket', 'bottle', 'wine glass', 'cup',
    'fork', 'knife', 'spoon', 'bowl', 'banana', 'apple',
    'sandwich', 'orange', 'broccoli', 'carrot', 'hot dog', 'pizza',
    'donut', 'cake', 'chair', 'couch', 'potted plant', 'bed',
    'dining table', 'toilet', 'tv', 'laptop', 'mouse', 'remote',
    'keyboard', 'cell phone', 'microwave', 'oven', 'toaster',
    'sink', 'refrigerator', 'book', 'clock', 'vase', 'scissors',
    'teddy bear', 'hair drier', 'toothbrush'
]




Configurations:
BACKBONE                       resnet101
BACKBONE_STRIDES               [4, 8, 16, 32, 64]
BATCH_SIZE                     1
BBOX_STD_DEV                   [0.1 0.1 0.2 0.2]
COMPUTE_BACKBONE_SHAPE         None
DETECTION_MAX_INSTANCES        100
DETECTION_MIN_CONFIDENCE       0.7
DETECTION_NMS_THRESHOLD        0.3
FPN_CLASSIF_FC_LAYERS_SIZE     1024
GPU_COUNT                      1
GRADIENT_CLIP_NORM             5.0
IMAGES_PER_GPU                 1
IMAGE_CHANNEL_COUNT            3
IMAGE_MAX_DIM                  1024
IMAGE_META_SIZE                93
IMAGE_MIN_DIM                  800
IMAGE_MIN_SCALE                0
IMAGE_RESIZE_MODE              square
IMAGE_SHAPE                    [1024 1024    3]
LEARNING_MOMENTUM              0.9
LEARNING_RATE                  0.001
LOSS_WEIGHTS                   {'rpn_class_loss': 1.0, 'rpn_bbox_loss': 1.0, 'mrcnn_class_loss': 1.0, 'mrcnn_bbox_loss': 1.0, 'mrcnn_mask_loss': 1.0}
MASK_POOL_SIZE                 14
MASK_SHAPE         

In [5]:
import matplotlib.pyplot as plt
import cv2
import random
import numpy as np
import pandas as pd
import math
import os
import pandas as pd
def HumanNum(results):
        r = results[0]
        HN = np.argwhere(r['class_ids']==1).shape[0]
        return HN
def HumanNum(results):
        r = results[0]
        HN = np.argwhere(r['class_ids']==1).shape[0]
        return HN
# Indicates # of human object detected
def HumanRatioBox(results):
        r = results[0]
        mask = r['masks']
        mask_areas_ratio = []
        imagearea = mask.shape[0] * mask.shape[1]
        for nmask in range(mask.shape[2]):
            tr_area=np.argwhere(mask[:,:,nmask]==True)
            if tr_area.sum() > 0:
                x_max=tr_area.max(axis=0)[0]
                x_min=tr_area.min(axis=0)[0]
                y_max=tr_area.max(axis=0)[1]
                y_min=tr_area.min(axis=0)[1]
                area = (x_max-x_min) * (y_max-y_min)
                mask_areas_ratio.append([area/imagearea])
            else:
                mask_areas_ratio.append([0])
        if np.argwhere(r['class_ids']==1).shape[0]>0:
            ra  = np.asarray(mask_areas_ratio)[np.reshape(np.argwhere(r['class_ids']==1),-1)]
            hr = np.reshape(ra,-1).sum()
        else:
            hr = 0  
        return hr
    
# compute the area of humans (sum of boxes)
#https://stackoverflow.com/questions/61246344/how-to-calculate-individual-mask-area-of-the-objects-in-mask-rcnn
def HumanRatioArea(results):
    r = results[0]
    mask = r['masks']
    mask_areas_ratio=[]
    imagearea = mask.shape[0] * mask.shape[1]
    for nmask in range(mask.shape[2]):
        area = mask[:,:,nmask].sum()
        mask_areas_ratio.append([area/imagearea])
    #print(mask_areas_ratio)
    if np.argwhere(r['class_ids']==1).shape[0]>0:
        ra  = np.asarray(mask_areas_ratio)[np.reshape(np.argwhere(r['class_ids']==1),-1)]
        hr = np.reshape(ra,-1).sum()
       # print(hr)
    else:
        hr = 0  
       # print(hr)
    return hr

In [6]:
#IMAGE_DIR = os.path.join(ROOT_DIR, "images")
# Image directory for all images & videos (v1)
VIDEO_DIR = os.path.join(ROOT_DIR, "Image_download_v2_post1-10000")
total = os.listdir(VIDEO_DIR)
len(total)
# only selecting .jpg files
videos = [i for i in total if '.mp4' in i]
import re

def atoi(text):
    return int(text) if text.isdigit() else text

def natural_keys(text):
    '''
    alist.sort(key=natural_keys) sorts in human order
    http://nedbatchelder.com/blog/200712/human_sorting.html
    (See Toothy's implementation in the comments)
    '''
    return [ atoi(c) for c in re.split(r'(\d+)', text) ]



videos.sort(key=natural_keys)
print(videos)
print(len(videos))

['6_1.mp4', '24.mp4', '24_1.mp4', '24_2.mp4', '24_4.mp4', '25_2.mp4', '30.mp4', '32.mp4', '33.mp4', '35_2.mp4', '36.mp4', '36_1.mp4', '37.mp4', '38_1.mp4', '41.mp4', '42.mp4', '43.mp4', '45_1.mp4', '48.mp4', '48_1.mp4', '54.mp4', '58.mp4', '59.mp4', '63.mp4', '65.mp4', '67.mp4', '80.mp4', '81.mp4', '99_2.mp4', '104_2.mp4', '106_3.mp4', '106_4.mp4', '108_1.mp4', '108_2.mp4', '108_3.mp4', '108_4.mp4', '109_2.mp4', '109_3.mp4', '116_1.mp4', '117_2.mp4', '119_1.mp4', '119_5.mp4', '121_2.mp4', '122_6.mp4', '136_6.mp4', '136_8.mp4', '136_9.mp4', '141_5.mp4', '150_3.mp4', '151_2.mp4', '153_1.mp4', '158.mp4', '179.mp4', '184.mp4', '189.mp4', '191.mp4', '192.mp4', '193.mp4', '195.mp4', '196.mp4', '202.mp4', '203.mp4', '205.mp4', '209.mp4', '213.mp4', '216.mp4', '217.mp4', '219.mp4', '222.mp4', '227.mp4', '229.mp4', '234.mp4', '239.mp4', '240.mp4', '245.mp4', '246.mp4', '249.mp4', '250.mp4', '251.mp4', '252.mp4', '264_1.mp4', '269.mp4', '271.mp4', '272.mp4', '273.mp4', '274.mp4', '275.mp4', '276

In [48]:
#loop through all the videos and process them
from datetime import datetime
csv_data = []
try:
    for i in videos:
        capture = cv2.VideoCapture(os.path.join(VIDEO_DIR, i))

        fps = capture.get(cv2.CAP_PROP_FPS)      # OpenCV2 version 2 used "CV_CAP_PROP_FPS"
        total_frames = int(capture.get(cv2.CAP_PROP_FRAME_COUNT))
        duration = total_frames/fps
        print(duration)


        frame_count = 0
        total_HumanNum=0
        total_HumanRatioArea=0
        total_HumanRatioBox=0

        start = datetime.now()
        while True:
            ret, frame = capture.read()
            # Bail out when the video file ends
            if not ret:
                break

            # Save each frame of the video to a list
            frame_count += 1

            print('name: {1}, frame_count :{0}'.format(frame_count,i))
            results = model.detect([frame])

            total_HumanNum+=HumanNum(results)
            total_HumanRatioArea+=HumanRatioArea(results)
            total_HumanRatioBox+=HumanRatioBox(results)
        #     print("HumanRatioArea:",HumanRatioArea(results))
        #     print("HumanRatioBox:",HumanRatioBox(results))
        #     print("HumanNum:",HumanNum(results))

        capture.release()
        avg_HumanNum=total_HumanNum/total_frames
        avg_HumanRatioArea=total_HumanRatioArea/total_frames
        avg_HumanRatioBox=total_HumanRatioBox/total_frames

        val = {
            'video_name': i,
            'video_length': duration,
            'total_frames': total_frames,
            'HumanNum': avg_HumanNum,
            'HumanRatioBox': avg_HumanRatioBox,
            'HumanRatioArea': avg_HumanRatioArea
        }
        csv_data.append(val)

        print(datetime.now()-start)
except:
    pass
    
csv_result = pd.DataFrame(csv_data)
csv_result.to_csv('result1.csv', index=False)

11.333333333333334
name: 25_2.mp4, frame_count :1
name: 25_2.mp4, frame_count :2
name: 25_2.mp4, frame_count :3
name: 25_2.mp4, frame_count :4
name: 25_2.mp4, frame_count :5
name: 25_2.mp4, frame_count :6
name: 25_2.mp4, frame_count :7
name: 25_2.mp4, frame_count :8
name: 25_2.mp4, frame_count :9
name: 25_2.mp4, frame_count :10
name: 25_2.mp4, frame_count :11
name: 25_2.mp4, frame_count :12
name: 25_2.mp4, frame_count :13
name: 25_2.mp4, frame_count :14
name: 25_2.mp4, frame_count :15
name: 25_2.mp4, frame_count :16
name: 25_2.mp4, frame_count :17
name: 25_2.mp4, frame_count :18
name: 25_2.mp4, frame_count :19
name: 25_2.mp4, frame_count :20
name: 25_2.mp4, frame_count :21
name: 25_2.mp4, frame_count :22
name: 25_2.mp4, frame_count :23
name: 25_2.mp4, frame_count :24
name: 25_2.mp4, frame_count :25
name: 25_2.mp4, frame_count :26
name: 25_2.mp4, frame_count :27
name: 25_2.mp4, frame_count :28
name: 25_2.mp4, frame_count :29
name: 25_2.mp4, frame_count :30
name: 25_2.mp4, frame_count :3

name: 25_2.mp4, frame_count :252
name: 25_2.mp4, frame_count :253
name: 25_2.mp4, frame_count :254
name: 25_2.mp4, frame_count :255
name: 25_2.mp4, frame_count :256
name: 25_2.mp4, frame_count :257
name: 25_2.mp4, frame_count :258
name: 25_2.mp4, frame_count :259
name: 25_2.mp4, frame_count :260
name: 25_2.mp4, frame_count :261
name: 25_2.mp4, frame_count :262
name: 25_2.mp4, frame_count :263
name: 25_2.mp4, frame_count :264
name: 25_2.mp4, frame_count :265
name: 25_2.mp4, frame_count :266
name: 25_2.mp4, frame_count :267
name: 25_2.mp4, frame_count :268
name: 25_2.mp4, frame_count :269
name: 25_2.mp4, frame_count :270
name: 25_2.mp4, frame_count :271
name: 25_2.mp4, frame_count :272
name: 25_2.mp4, frame_count :273
name: 25_2.mp4, frame_count :274
name: 25_2.mp4, frame_count :275
name: 25_2.mp4, frame_count :276
name: 25_2.mp4, frame_count :277
name: 25_2.mp4, frame_count :278
name: 25_2.mp4, frame_count :279
name: 25_2.mp4, frame_count :280
name: 25_2.mp4, frame_count :281
name: 25_2

name: 30.mp4, frame_count :173
name: 30.mp4, frame_count :174
name: 30.mp4, frame_count :175
name: 30.mp4, frame_count :176
name: 30.mp4, frame_count :177
name: 30.mp4, frame_count :178
name: 30.mp4, frame_count :179
name: 30.mp4, frame_count :180
name: 30.mp4, frame_count :181
name: 30.mp4, frame_count :182
name: 30.mp4, frame_count :183
name: 30.mp4, frame_count :184
name: 30.mp4, frame_count :185
name: 30.mp4, frame_count :186
name: 30.mp4, frame_count :187
name: 30.mp4, frame_count :188
name: 30.mp4, frame_count :189
name: 30.mp4, frame_count :190
name: 30.mp4, frame_count :191
name: 30.mp4, frame_count :192
name: 30.mp4, frame_count :193
name: 30.mp4, frame_count :194
name: 30.mp4, frame_count :195
name: 30.mp4, frame_count :196
name: 30.mp4, frame_count :197
name: 30.mp4, frame_count :198
name: 30.mp4, frame_count :199
name: 30.mp4, frame_count :200
name: 30.mp4, frame_count :201
name: 30.mp4, frame_count :202
name: 30.mp4, frame_count :203
name: 30.mp4, frame_count :204
name: 30

name: 30.mp4, frame_count :438
name: 30.mp4, frame_count :439
name: 30.mp4, frame_count :440
name: 30.mp4, frame_count :441
name: 30.mp4, frame_count :442
name: 30.mp4, frame_count :443
name: 30.mp4, frame_count :444
name: 30.mp4, frame_count :445
name: 30.mp4, frame_count :446
name: 30.mp4, frame_count :447
name: 30.mp4, frame_count :448
name: 30.mp4, frame_count :449
name: 30.mp4, frame_count :450
name: 30.mp4, frame_count :451
name: 30.mp4, frame_count :452
name: 30.mp4, frame_count :453
name: 30.mp4, frame_count :454
name: 30.mp4, frame_count :455
name: 30.mp4, frame_count :456
name: 30.mp4, frame_count :457
name: 30.mp4, frame_count :458
name: 30.mp4, frame_count :459
name: 30.mp4, frame_count :460
name: 30.mp4, frame_count :461
name: 30.mp4, frame_count :462
name: 30.mp4, frame_count :463
name: 30.mp4, frame_count :464
name: 30.mp4, frame_count :465
name: 30.mp4, frame_count :466
name: 30.mp4, frame_count :467
name: 30.mp4, frame_count :468
name: 30.mp4, frame_count :469
name: 30

name: 30.mp4, frame_count :703
name: 30.mp4, frame_count :704
name: 30.mp4, frame_count :705
name: 30.mp4, frame_count :706
name: 30.mp4, frame_count :707
name: 30.mp4, frame_count :708
name: 30.mp4, frame_count :709
name: 30.mp4, frame_count :710
name: 30.mp4, frame_count :711
name: 30.mp4, frame_count :712
name: 30.mp4, frame_count :713
name: 30.mp4, frame_count :714
name: 30.mp4, frame_count :715
name: 30.mp4, frame_count :716
name: 30.mp4, frame_count :717
name: 30.mp4, frame_count :718
name: 30.mp4, frame_count :719
name: 30.mp4, frame_count :720
name: 30.mp4, frame_count :721
name: 30.mp4, frame_count :722
name: 30.mp4, frame_count :723
name: 30.mp4, frame_count :724
name: 30.mp4, frame_count :725
name: 30.mp4, frame_count :726
name: 30.mp4, frame_count :727
name: 30.mp4, frame_count :728
name: 30.mp4, frame_count :729
name: 30.mp4, frame_count :730
name: 30.mp4, frame_count :731
name: 30.mp4, frame_count :732
name: 30.mp4, frame_count :733
name: 30.mp4, frame_count :734
name: 30

name: 30.mp4, frame_count :968
name: 30.mp4, frame_count :969
name: 30.mp4, frame_count :970
name: 30.mp4, frame_count :971
name: 30.mp4, frame_count :972
name: 30.mp4, frame_count :973
name: 30.mp4, frame_count :974
name: 30.mp4, frame_count :975
name: 30.mp4, frame_count :976
name: 30.mp4, frame_count :977
name: 30.mp4, frame_count :978
name: 30.mp4, frame_count :979
name: 30.mp4, frame_count :980
name: 30.mp4, frame_count :981
name: 30.mp4, frame_count :982
name: 30.mp4, frame_count :983
name: 30.mp4, frame_count :984
name: 30.mp4, frame_count :985
name: 30.mp4, frame_count :986
name: 30.mp4, frame_count :987
name: 30.mp4, frame_count :988
name: 30.mp4, frame_count :989
name: 30.mp4, frame_count :990
name: 30.mp4, frame_count :991
name: 30.mp4, frame_count :992
name: 30.mp4, frame_count :993
name: 30.mp4, frame_count :994
name: 30.mp4, frame_count :995
name: 30.mp4, frame_count :996
name: 30.mp4, frame_count :997
name: 30.mp4, frame_count :998
name: 30.mp4, frame_count :999
name: 30

name: 30.mp4, frame_count :1226
name: 30.mp4, frame_count :1227
name: 30.mp4, frame_count :1228
name: 30.mp4, frame_count :1229
name: 30.mp4, frame_count :1230
name: 30.mp4, frame_count :1231
name: 30.mp4, frame_count :1232
name: 30.mp4, frame_count :1233
name: 30.mp4, frame_count :1234
name: 30.mp4, frame_count :1235
name: 30.mp4, frame_count :1236
name: 30.mp4, frame_count :1237
name: 30.mp4, frame_count :1238
name: 30.mp4, frame_count :1239
name: 30.mp4, frame_count :1240
name: 30.mp4, frame_count :1241
name: 30.mp4, frame_count :1242
name: 30.mp4, frame_count :1243
name: 30.mp4, frame_count :1244
name: 30.mp4, frame_count :1245
name: 30.mp4, frame_count :1246
name: 30.mp4, frame_count :1247
name: 30.mp4, frame_count :1248
name: 30.mp4, frame_count :1249
name: 30.mp4, frame_count :1250
name: 30.mp4, frame_count :1251
name: 30.mp4, frame_count :1252
name: 30.mp4, frame_count :1253
name: 30.mp4, frame_count :1254
name: 30.mp4, frame_count :1255
name: 30.mp4, frame_count :1256
name: 30

name: 30.mp4, frame_count :1483
name: 30.mp4, frame_count :1484
name: 30.mp4, frame_count :1485
name: 30.mp4, frame_count :1486
name: 30.mp4, frame_count :1487
name: 30.mp4, frame_count :1488
name: 30.mp4, frame_count :1489
name: 30.mp4, frame_count :1490
name: 30.mp4, frame_count :1491
name: 30.mp4, frame_count :1492
name: 30.mp4, frame_count :1493
name: 30.mp4, frame_count :1494
name: 30.mp4, frame_count :1495
name: 30.mp4, frame_count :1496
name: 30.mp4, frame_count :1497
name: 30.mp4, frame_count :1498
name: 30.mp4, frame_count :1499
name: 30.mp4, frame_count :1500
name: 30.mp4, frame_count :1501
name: 30.mp4, frame_count :1502
name: 30.mp4, frame_count :1503
name: 30.mp4, frame_count :1504
name: 30.mp4, frame_count :1505
name: 30.mp4, frame_count :1506
name: 30.mp4, frame_count :1507
name: 30.mp4, frame_count :1508
name: 30.mp4, frame_count :1509
name: 30.mp4, frame_count :1510
name: 30.mp4, frame_count :1511
name: 30.mp4, frame_count :1512
name: 30.mp4, frame_count :1513
name: 30

name: 30.mp4, frame_count :1740
name: 30.mp4, frame_count :1741
name: 30.mp4, frame_count :1742
name: 30.mp4, frame_count :1743
name: 30.mp4, frame_count :1744
name: 30.mp4, frame_count :1745
name: 30.mp4, frame_count :1746
name: 30.mp4, frame_count :1747
name: 30.mp4, frame_count :1748
name: 30.mp4, frame_count :1749
name: 30.mp4, frame_count :1750
name: 30.mp4, frame_count :1751
name: 30.mp4, frame_count :1752
name: 30.mp4, frame_count :1753
name: 30.mp4, frame_count :1754
name: 30.mp4, frame_count :1755
name: 30.mp4, frame_count :1756
name: 30.mp4, frame_count :1757
name: 30.mp4, frame_count :1758
name: 30.mp4, frame_count :1759
name: 30.mp4, frame_count :1760
name: 30.mp4, frame_count :1761
name: 30.mp4, frame_count :1762
name: 30.mp4, frame_count :1763
name: 30.mp4, frame_count :1764
name: 30.mp4, frame_count :1765
name: 30.mp4, frame_count :1766
name: 30.mp4, frame_count :1767
name: 30.mp4, frame_count :1768
name: 30.mp4, frame_count :1769
name: 30.mp4, frame_count :1770
name: 30

name: 32.mp4, frame_count :204
name: 32.mp4, frame_count :205
name: 32.mp4, frame_count :206
name: 32.mp4, frame_count :207
name: 32.mp4, frame_count :208
name: 32.mp4, frame_count :209
name: 32.mp4, frame_count :210
name: 32.mp4, frame_count :211
name: 32.mp4, frame_count :212
name: 32.mp4, frame_count :213
name: 32.mp4, frame_count :214
name: 32.mp4, frame_count :215
name: 32.mp4, frame_count :216
name: 32.mp4, frame_count :217
name: 32.mp4, frame_count :218
name: 32.mp4, frame_count :219
name: 32.mp4, frame_count :220
name: 32.mp4, frame_count :221
name: 32.mp4, frame_count :222
name: 32.mp4, frame_count :223
name: 32.mp4, frame_count :224
name: 32.mp4, frame_count :225
name: 32.mp4, frame_count :226
name: 32.mp4, frame_count :227
name: 32.mp4, frame_count :228
name: 32.mp4, frame_count :229
name: 32.mp4, frame_count :230
name: 32.mp4, frame_count :231
name: 32.mp4, frame_count :232
name: 32.mp4, frame_count :233
name: 32.mp4, frame_count :234
name: 32.mp4, frame_count :235
name: 32